In [ ]:
# import os
# from openai import AzureOpenAI

# endpoint = "https://my-openai-service-capstone.openai.azure.com/"
# model_name = "gpt-4o"
# deployment = "gpt-4o-2"

# subscription_key = "<your-api-key>"
# api_key = "Ab2VvoYKqMSDw8YutIW0s8kcuBqlDTKQCqAIydVqDhVxoerl4edeJQQJ99BFACYeBjFXJ3w3AAABACOGk4rz"
# api_version = "2024-12-01-preview"

# client = AzureOpenAI(
#     api_version=api_version,
#     azure_endpoint=endpoint,
#     api_key=api_key,
# )

# response = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a helpful assistant.",
#         },
#         {
#             "role": "user",
#             "content": "I am going to Paris, what should I see?",
#         }
#     ],
#     max_tokens=4096,
#     temperature=1.0,
#     top_p=1.0,
#     model=deployment
# )

# print(response.choices[0].message.content)

In [ ]:
# from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
# import semantic_kernel as sk
# endpoint = "https://my-openai-service-capstone.openai.azure.com/"
# model_name = "gpt-4o"
# deployment = "gpt-4o-2"

# subscription_key = "<your-api-key>"
# api_key = "Ab2VvoYKqMSDw8YutIW0s8kcuBqlDTKQCqAIydVqDhVxoerl4edeJQQJ99BFACYeBjFXJ3w3AAABACOGk4rz"
# api_version = "2024-12-01-preview"

# kernel = sk.Kernel()

# kernel.add_chat_service("dv", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key))

# prompt = kernel.create_semantic_function("""
# Write a simple palindrome python program.
# """)

# # Run your prompt
# print(prompt()) 

In [1]:
ProgamManager = """You are a program manager which will take the requirement and create a plan for creating app. Program Manager understands the user requirements and form the detail documents 
with requirements and costing. """
SoftwareEngineer = """You are Software Engieer, and your goal is develop web app using HTML and JavaScript (JS) by taking into consideration all the requirements given by Program Manager. """
Manager = """You are manager which will review software engineer code, and make sure all client requirements are completed.Once all client requirements are completed, you can approve the request by 
just responding "approve" """

REVIEWER_NAME = "ProgamManager_agent"
WRITER_NAME = "software_engineer_agent"
SAVER_NAME = "product_owner_agent"

In [2]:
from semantic_kernel import Kernel
import os
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.functions import KernelFunctionFromPrompt, kernel_function
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from dotenv import load_dotenv
load_dotenv()

class FileSavePlugin:
    """ A Plugin used for saving files."""
    # Function to save content to a file
    @kernel_function(description="saves file to disk", name="save_to_file")
    async def save_to_file(self, content: str):
        file_path = "output.txt"
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)
        return f"Content saved to {file_path}"

# Initialize the kernel
kernel = Kernel()
chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    )
kernel.add_service(chat_completion_service)

 # Get the AI Service settings
settings = kernel.get_prompt_execution_settings_from_service_id("default")

# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Add the Plugin to the Kernel
kernel.add_plugin(FileSavePlugin(), plugin_name="FileSavePlugin")
# Create agents
ProgamManager_agent = ChatCompletionAgent(
    name="ProgamManager_agent",
    instructions=ProgamManager,
    kernel=kernel
)
print("ProgamManager Agent Initialized")

software_engineer_agent = ChatCompletionAgent(
    name="software_engineer_agent",
    instructions=SoftwareEngineer,
    kernel=kernel
)
print("Software Engineer Agent Initialized")

product_owner_agent = ChatCompletionAgent(
    name="product_owner_agent",
    instructions=Manager,
    kernel=kernel
)
print("Product Owner Agent Initialized")

ProgamManager Agent Initialized
Software Engineer Agent Initialized
Product Owner Agent Initialized


In [3]:
 # Define a selection function to determine which agent should take the next turn.
selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.
Never choose the participant named in the RESPONSE.

Choose only from these participants:
- {REVIEWER_NAME}
- {WRITER_NAME}
- {SAVER_NAME}

Rules:
- If RESPONSE is user input, it is {REVIEWER_NAME}'s turn.
- If RESPONSE is by {REVIEWER_NAME}, it is {WRITER_NAME}'s turn.
- If RESPONSE is by {WRITER_NAME}, it is {SAVER_NAME}'s turn.
- If RESPONSE is by {SAVER_NAME}, the process is complete.

RESPONSE:
{{{{$lastmessage}}}}
""",
    )

    # Define a termination function where the process ends after the saver agent confirms.
termination_keyword = "approve"

termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt=f"""
Examine the RESPONSE and determine whether the content has been saved.
If the content has been saved, respond with a single word without explanation: {termination_keyword}.

RESPONSE:
{{{{$lastmessage}}}}
""",
    )

In [4]:
from semantic_kernel.contents import ChatHistoryTruncationReducer

history_reducer = ChatHistoryTruncationReducer(target_count=5)

# Create the AgentGroupChat with selection and termination strategies.
chat = AgentGroupChat(
    agents=[ProgamManager_agent, software_engineer_agent, product_owner_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=ProgamManager_agent,
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
        history_variable_name="lastmessage",
        history_reducer=history_reducer,
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[product_owner_agent],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)

print(
    "Ready! Type your input, or 'exit' to quit, 'reset' to restart the conversation."
)

Ready! Type your input, or 'exit' to quit, 'reset' to restart the conversation.


In [6]:
user_input = """I want to develop calculator app. Keep it very simple. And get final approval from manager."""
chat.add_chat_message(message=user_input)

<coroutine object AgentChat.add_chat_message at 0x0000020D40FCC430>

In [9]:
help(chat.add_chat_message)

Help on method add_chat_message in module semantic_kernel.agents.group_chat.agent_chat:

async add_chat_message(message: str | semantic_kernel.contents.chat_message_content.ChatMessageContent) -> None method of semantic_kernel.agents.group_chat.agent_group_chat.AgentGroupChat instance
    Add a chat message.



In [ ]:
is_complete = False
while not is_complete:
    print()
    user_input = input("User > ").strip()
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    if user_input.lower() == "reset":
        await chat.reset()
        print("[Conversation has been reset]")
        continue

    # Add the current user_input to the chat
    await chat.add_chat_message(message=user_input)

    try:
        async for response in chat.invoke():
            if response is None or not response.name:
                continue
            print()
            print(f"# {response.name.upper()}:\n{response.content}")
    except Exception as e:
        print(f"Error during chat invocation: {e}")

    # Reset the chat's complete flag for the new conversation round.
    chat.is_complete = False

In [ ]:
# Add the current user_input to the chat
user_input = """I want to develop calculator app. Keep it very simple. And get final approval from manager."""
await chat.add_chat_message(message=user_input)
async for response in chat.invoke():
        if response is None or not response.name:
                continue
        print()
        print(f"# {response.name.upper()}:\n{response.content}")

In [ ]:
import os
import asyncio
import re
import subprocess
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from dotenv import load_dotenv
from semantic_kernel.functions import KernelFunctionFromPrompt, kernel_function
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
load_dotenv()

# Define agent names
BA = "BusinessAnalyst"
SE = "SoftwareEngineer"
PO = "ProductOwner"

class FileSavePlugin:
    """A Plugin used for saving HTML content to a file."""

    @kernel_function(description="Extracts HTML content and saves it to a file", name="extract_and_save_html")
    async def extract_and_save_html(self, content: str):
        # Define a regular expression pattern to match HTML content
        html_pattern = re.compile(r'<html.*?>.*?</html>', re.DOTALL | re.IGNORECASE)

        # Extract HTML content using the pattern
        match = html_pattern.search(content)
        if match:
            html_content = match.group(0)
            file_path = "index.html"
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(html_content)
            # # Push to GitHub
            # subprocess.run(['bash', 'push_to_github.sh'], check=True)
            return f"HTML content saved to {file_path}"
        else:
            return "No HTML content found in the provided text."

def initialize_kernel():
    kernel = Kernel()
    #Challene 02 - Add Kernel
    chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    )
    kernel.add_service(chat_completion_service)
    return kernel

# Define a custom termination strategy
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""
    
    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        for message in history:
            if "APPROVED" in message.content:
                return True
        return False

# Define agent personas
business_analyst_persona = """
    You are a Business Analyst which will take the requirements from the user (also known as a 'customer') 
    and create a project plan for creating the requested app. The Business Analyst understands the user 
    requirements and creates detailed documents with requirements and costing. The documents should be 
    usable by the SoftwareEngineer as a reference for implementing the required features, and by the 
    Product Owner for reference to determine if the application delivered by the Software Engineer meets 
    all of the user's requirements.
    """

software_engineer_persona = """
    You are a Software Engineer, and your goal is create a web app using HTML and JavaScript by taking 
    into consideration all the requirements given by the Business Analyst. The application should 
    implement all the requested features. Deliver the code to the Product Owner for review when completed. You can also ask questions of the BusinessAnalyst to clarify any requirements that are unclear.
    """

product_owner_persona = """
    You are the Product Owner which will review the software engineer's code to ensure all user requirements 
    are completed. You are the guardian of quality, ensuring the final product meets all specifications. 
    IMPORTANT: Verify that the Software Engineer has shared the HTML code using the format ```html [code] ```. 
    This format is required for the code to be saved and pushed to GitHub. Once all client requirements 
    are completed and the code is properly formatted, reply with 'READY FOR USER APPROVAL'. If there are 
    missing features or formatting issues, you will need to send a request back to the SoftwareEngineer 
    or BusinessAnalyst with details of the defect.
    """

async def run_multi_agent():
    """Implement the multi-agent system."""
    # Initialize the Kernel
    kernel = initialize_kernel()

    # Get the AI Service settings
    settings = kernel.get_prompt_execution_settings_from_service_id("default")

    # Configure the function choice behavior to auto invoke kernel functions
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    # Add the Plugin to the Kernel
    kernel.add_plugin(FileSavePlugin(), plugin_name="FileSavePlugin")
   
    # Create agents
    business_analyst_agent = ChatCompletionAgent(
        name=BA,
        instructions=business_analyst_persona,
        kernel=kernel
    )
    print("Business Analyst Agent Initialized")

    software_engineer_agent = ChatCompletionAgent(
        name=SE,
        instructions=software_engineer_persona,
        kernel=kernel
    )
    print("Software Engineer Agent Initialized")

    product_owner_agent = ChatCompletionAgent(
        name=PO,
        instructions=product_owner_persona,
        kernel=kernel
    )
    print("Product Owner Agent Initialized")

    # Define a selection function to determine which agent should take the next turn.
    selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""Examine the provided RESPONSE and choose the next participant.
        State only the name of the chosen participant without explanation.
        Never choose the participant named in the RESPONSE.

        Choose only from these participants:
        - {BA}
        - {SE}
        - {PO}

        Rules:
        - If RESPONSE is user input, it is {BA}'s turn.
        - If RESPONSE is by {BA}, it is {SE}'s turn.
        - If RESPONSE is by {SE}, it is {PO}'s turn.
        - If RESPONSE is by {PO}, the process is complete.

        RESPONSE:
        {{{{$lastmessage}}}}
        """,
            )
    
    # Define a termination function where the process ends after the saver agent confirms.
    termination_keyword = "APPROVED"

    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt=f"""
            Examine the RESPONSE and determine whether the content has been APPROVED.
            If the content has been saved, respond with a single word without explanation: {termination_keyword}.

            RESPONSE:
            {{{{$lastmessage}}}}
        """,)

    history_reducer = ChatHistoryTruncationReducer(target_count=3)

    # Create an AgentGroupChat with the agents
    group_chat = AgentGroupChat(
        agents=[business_analyst_agent, software_engineer_agent, product_owner_agent],
        selection_strategy=KernelFunctionSelectionStrategy(
            initial_agent=business_analyst_agent,
            function=selection_function,
            kernel=kernel,
            result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else SE,
            history_variable_name="lastmessage",
            history_reducer=history_reducer,
        ),
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[product_owner_agent],
            function=termination_function,
            kernel=kernel,
            result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
            history_variable_name="lastmessage",
            maximum_iterations=5,
            history_reducer=history_reducer,
        ),
    )

    is_complete = False
    while not is_complete:
        print(
            "Ready! Type your input, or 'exit' to quit, 'reset' to restart the conversation."
        )
        user_input = input("User > ").strip()

        if user_input.lower() == "exit":
            is_complete = True
            print("Exiting conversation.")
            break

        if user_input.lower() == "reset":
            await group_chat.reset()
            print("[Conversation has been reset]")
            continue

        # Add the current user_input to the chat
        await group_chat.add_chat_message(message=user_input)

        try:
            async for response in group_chat.invoke():
                if response is None or not response.name:
                    continue
                print()
                print(f"# {response.name.upper()}:\n{response.content}")

                # Check for approval keyword
                if "READY FOR USER APPROVAL" in response.content.upper():
                    print("Approval Detected")
                    
                    is_complete = True
                    print("Conversation completed.")
                    break
        except:
            print("Did not work properly")

# I want to develop calculator app. Keep it very simple. And get final approval from manager.

In [ ]:
await run_multi_agent()